In [ ]:
from colmap_testing.colmap_helpers import read_write_model
import os
import cv2
import numpy as np
from constants import *
from matplotlib import pyplot as plt
import open3d as o3d
import plotly

In [ ]:
recording_id = "ff0acdab-123d-41d8-bfd6-b641f99fc8eb"

pointcloud_depth_camera_path = os.path.join(recordings_folder, recording_id, "depth_camera_pointcloud.ply")
meshed_depth_camera_path = os.path.join(recordings_folder, recording_id,"meshed_depth_camera.ply")
pointcloud_colmap_path =os.path.join(recordings_folder,"/scratch_net/snapo/mlindner/docs/gaze_data_acquisition/mathis/recordings/ff0acdab-123d-41d8-bfd6-b641f99fc8eb/colmap_ws/automatic_recontructor/dense/0/meshed-poisson.ply")

In [ ]:
bag_file_path = os.path.join(recordings_folder, recording_id, "realsensed435.bag")
# First import the library
import pyrealsense2 as rs


# Declare pointcloud object, for calculating pointclouds and texture mappings
pc = rs.pointcloud()
# We want the points object to be persistent so we can display the last cloud when a frame drops
points = rs.points()

# Declare RealSense pipeline, encapsulating the actual device and sensors
pipe = rs.pipeline()
config = rs.config()
# Enable depth stream
#config.enable_stream(rs.stream.depth)
config.enable_device_from_file(bag_file_path, repeat_playback=False)


# Start streaming with chosen configuration
profile = pipe.start(config)
playback = profile.get_device().as_playback()
playback.set_real_time(False)
# We'll use the colorizer to generate texture for our PLY
# (alternatively, texture can be obtained from color or infrared stream)
colorizer = rs.colorizer()

try:
    # Wait for the next set of frames from the camera
    frames = pipe.wait_for_frames()
    colorized = colorizer.process(frames)

    # Create save_to_ply object
    ply = rs.save_to_ply(pointcloud_depth_camera_path)

    # Set options to the desired values
    # In this example we'll generate a textual PLY with normals (mesh is already created by default)
    ply.set_option(rs.save_to_ply.option_ply_binary, False)
    ply.set_option(rs.save_to_ply.option_ply_normals, True)
    ply.set_option(rs.save_to_ply.option_ply_mesh, False)

    print("Saving to 1.ply...")
    # Apply the processing block to the frameset which contains the depth frame and the texture
    ply.process(colorized)
    print("Done")
finally:
    pipe.stop()

In [ ]:
recording = "/scratch_net/snapo/mlindner/docs/gaze_data_acquisition/mathis/recordings/ff0acdab-123d-41d8-bfd6-b641f99fc8eb"

pointcloud_depth_camera = o3d.io.read_point_cloud(os.path.join(recording,"depth_camera_pointcloud.ply"))
pointcloud_colmap = o3d.io.read_point_cloud(os.path.join(recording,"/scratch_net/snapo/mlindner/docs/gaze_data_acquisition/mathis/recordings/ff0acdab-123d-41d8-bfd6-b641f99fc8eb/colmap_ws/automatic_recontructor/dense/0/fused.ply"))

# Extract key features
keypoints1 = np.asarray(pointcloud_depth_camera.points)
keypoints2 = np.asarray(pointcloud_colmap.points)




#↨inital guess: x_rotation = 180°, y_rotation = 0°, z_rotation = 0°, x_translation = 0, y_translation = 0, z_translation = 0, scale = 4
initial_matrix = np.array([[4, 0, 0, 0],
                           [0, -4, 0, 0],
                           [0, 0, -4, 0],
                           [0, 0, 0, 0]])

# Use Iterative Closest Point algorithm to find the best alignment
reg_p2p = o3d.pipelines.registration.registration_icp(pointcloud_depth_camera, pointcloud_colmap, max_correspondence_distance = 0.5, init = initial_matrix)


transformation = reg_p2p.transformation
print(transformation)
# Apply the transformation matrix to the second point cloud

keypoints1 = keypoints1 @ transformation[:3,:3].T + transformation[:3,3]




